# Create forcing and static data for CREDIT models 

In [1]:
import os
import sys
import glob
import numpy as np
from datetime import datetime, timedelta 

import zarr
import xarray as xr
import netCDF4 as nc

## Forcing dataset

### Import TSI from source

In [3]:
TOA_forcing_path = '/glade/derecho/scratch/dgagne/credit_scalers/solar_radiation_2024-03-29_0204.nc' 

with nc.Dataset(TOA_forcing_path, 'r') as ncio:
    TSI = ncio['tsi'][...]
    TSI_time = ncio['time'][...]
    latitude = ncio['latitude'][...]
    longitude = ncio['longitude'][...]

### Create datetime reference

In [4]:
base = datetime(2000, 1, 1, 0) # using the year 2000 as base values
date_list = [base + timedelta(hours=h) for h in range(len(TSI_time))]
time = np.array(date_list, dtype='datetime64').astype(dtype='datetime64[ns]')

### Create and save the dataset

In [5]:
data_forcing = xr.Dataset(
    {
    'TSI': xr.DataArray(data=TSI,
                        dims=['time', 'latitude', 'longitude'],
                        coords = {'time':time, 'latitude': latitude, 'longitude':longitude, },
                        attrs  = {'units': 'J m-2'}
                       ),
    },
    attrs = {'variables': 'Total Solar Irradiance (TSI)'}
    )

In [8]:
save_name = '/glade/campaign/cisl/aiml/ksha/CREDIT/forcing.nc'
data_forcing.to_netcdf(save_name, engine='netcdf4', encoding=None)

### Read the data

In [9]:
xr.open_dataset(save_name)

<xarray.Dataset> Size: 29GB
Dimensions:    (time: 8784, latitude: 640, longitude: 1280)
Coordinates:
  * time       (time) datetime64[ns] 70kB 2000-01-01 ... 2000-12-31T23:00:00
  * latitude   (latitude) float64 5kB 89.78 89.51 89.23 ... -89.23 -89.51 -89.78
  * longitude  (longitude) float64 10kB 0.0 0.2812 0.5625 ... 359.2 359.4 359.7
Data variables:
    TSI        (time, latitude, longitude) float32 29GB ...
Attributes:
    variables:  Total Solar Irradiance (TSI)

## Static variables

### Import LSM and Z_GDS4_SFC from source

In [10]:
latitude_weights = '/glade/u/home/wchapman/MLWPS/DataLoader/LSM_static_variables_ERA5_zhght.nc'

with nc.Dataset(latitude_weights, 'r') as ncio:
    Z_GDS4_SFC = ncio['Z_GDS4_SFC'][...]
    LSM = ncio['LSM'][...]
    latitude = ncio['latitude'][...]
    longitude = ncio['longitude'][...]

### Create and save the dataset

In [12]:
data_static = xr.Dataset(
    {
    'Z_GDS4_SFC': xr.DataArray(data=Z_GDS4_SFC,
                               dims=['latitude', 'longitude'],
                               coords = {'latitude': latitude, 'longitude':longitude},
                               attrs  = {'units': 'm**2 s**-2'}
                              ),
    'LSM': xr.DataArray(data=LSM[0, ...],
                        dims=['latitude', 'longitude'],
                        coords = {'latitude': latitude, 'longitude':longitude},
                        attrs  = {'units': 'none'}
                       )
    },
    attrs = {'variables': 'Geopotential relative to the mean sea level (Z_GDS4_SFC); Land-Sea Mask (LSM)'}
    )

In [13]:
save_name = '/glade/campaign/cisl/aiml/ksha/CREDIT/static.nc'
data_static.to_netcdf(save_name, engine='netcdf4', encoding=None)

### Read the data

In [14]:
xr.open_dataset(save_name)

<xarray.Dataset> Size: 10MB
Dimensions:     (latitude: 640, longitude: 1280)
Coordinates:
  * latitude    (latitude) float64 5kB 89.78 89.51 89.23 ... -89.51 -89.78
  * longitude   (longitude) float64 10kB 0.0 0.2812 0.5625 ... 359.2 359.4 359.7
Data variables:
    Z_GDS4_SFC  (latitude, longitude) float64 7MB ...
    LSM         (latitude, longitude) float32 3MB ...
Attributes:
    variables:  Geopotential relative to the mean sea level (Z_GDS4_SFC); Lan...